In [ ]:
from   astropy import table, time
from   astropy.io import fits
import json
import os
from   plotsettings import *
import requests
from   scipy import interpolate
from   standard_libraries import *

In [ ]:
def api(method, endpoint, data=None):
    """API request"""

    headers = {'Authorization': f'token {myToken}'}
    print(headers)
    response = requests.request(method, endpoint, json=data, headers=headers)
    print(f'HTTP code: {response.status_code}, {response.reason}')
    return response

def get_groups(source):
    """Get the groups a source belongs to"""

    response = api('GET',
                   f'https://fritz.science/api/sources/{source}'
                   )

    if response.status_code == 200:
        groups = response.json()['data']['groups']
    
    else:
        print(f'HTTP code: {response.status_code}, {response.reason}')

    return groups

In [ ]:
root = os.path.expanduser('~') + '/NOT/'
#root = '/home/steve/Downloads/'
#root = '/Volumes/Work/pynot/'
path = root + 'raw/'

In [ ]:
date = '2023-07-23'

In [ ]:
os.chdir(path+date+'/spectra/')

In [ ]:
!ls ./ | grep -v fits

In [ ]:
ls ZTF23aaltxkt/al-gr4_slit1.0/ob1/

In [ ]:
try:
    del file
except:
    pass

try:
    del file1
except:
    pass

try:
    del file2
except:
    pass

try:
    del file2
except:
    pass

try:
    del file3
except:
    pass

file1            = 'ZTF23aaltxkt/al-gr4_slit1.0/ob1/FLUX1D_ZTF23aaltxkt.fits'
file2            = 'ZTF23aaltxkt/al-gr4_slit1.0/ob2/FLUX1D_ZTF23aaltxkt.fits'
#file3            = 'ZTF22abnvurz/al-gr4_slit1.0/ob3/FLUX1D_ZTF22abnvurz.fits'
#file3            = 'ZTF22aafrjnw/al-gr4_slit1.3/ob3/FLUX1D_ZTF22aafrjnw.fits'

# If you extracted spectra of multiple traces, use the option 'hdu' to select the relevant column

epochs           = {}
epochs['Epoch 1'] = table.Table.read(file1, hdu=2)
#epochs['Epoch 1'] = epochs['Epoch 1'][ (epochs['Epoch 1']['WAVE'] > 3500) & (epochs['Epoch 1']['WAVE'] < 9000)]
#epochs['Epoch 2'] = table.Table.read(file2, hdu=1)
epochs['Epoch 2'] = table.Table.read(file2, hdu=2)#, hdu='OBJ1')

In [ ]:
epochs['Epoch 1']

In [ ]:
hdu=fits.open(file1)
hdu.info()
#hdu.header

In [ ]:
# Compute the fudge factor to co-add multiple epochs

In [ ]:
ref_wave    = epochs['Epoch 1']['WAVE']
ref_vals    = epochs['Epoch 1']['FLUX']
ref_errs    = epochs['Epoch 1']['ERR']

for ii, epoch in enumerate(epochs):

    if ii > 0:
        
        temp_wave        = epochs[epoch]['WAVE']
        temp_vals        = epochs[epoch]['FLUX']
        temp_errs        = epochs[epoch]['ERR']
        
        temp_vals_interp = interpolate.interp1d(temp_wave, temp_vals,    bounds_error=False, fill_value=np.nan)
        temp_errs_interp = interpolate.interp1d(temp_wave, temp_errs,    bounds_error=False, fill_value=np.nan)

        ratio            = ref_vals / temp_vals_interp(ref_wave)
        ratio_err        = ratio * np.sqrt((temp_errs_interp(ref_wave)/temp_vals_interp(ref_wave))**2 + (ref_errs / ref_vals)**2)
        ratio_weights    = 1/ratio_err*2
        
        
        mask             = np.where( (np.logical_not(np.isnan(ratio*ratio_weights))) & (ref_wave > 4000) & (ref_wave < 9000))[0]
        
        fudge_weighted_mean = np.sum((ratio * ratio_weights)[mask]) / np.sum(ratio_weights[mask])
        print(fudge_weighted_mean)

        epochs[epoch]['FLUX'] *= fudge_weighted_mean
        epochs[epoch]['ERR']  *= fudge_weighted_mean

In [ ]:
# Get FITS header

In [ ]:
header = fits.getheader(file1, hdu=0)

In [ ]:
# Create a new header with a summary of the observations

In [ ]:
comments               = {}
comments['OBJECT']     = header['TCSTGT']
comments['RA']         = header['RA']
comments['DEC']        = header['DEC']
comments['OBSERVER']   = 'Jesper Sollerman, Steve Schulze'
comments['REDUCER']    = 'Steve Schulze'
comments['PIPELINE']   = 'PyNOT v{}'.format(hdu[1].header['AUTHOR'].split(' ')[-1])
comments['DATE-OBS']   = header['DATE-OBS']
comments['PROPID']     = header['PROPID']
comments['JD']         = np.round(time.Time(comments['DATE-OBS'], format='isot', scale='utc').jd, 5)
comments['MJD']        = np.round(time.Time(comments['DATE-OBS'], format='isot', scale='utc').mjd, 5)
comments['EXPTIME']    = header['EXPTIME'] * len(epochs.keys())
comments['TELESCOPE']  = header['TELESCOP']
comments['INSTRUMENT'] = header['INSTRUME'].split('_')[0]
comments['SLIT']       = header['ALAPRTNM']
comments['DISERPER']   = header['ALGRNM']
comments['WLENSYSTEM'] = 'vacuum'

In [ ]:
comments

In [ ]:
if len(epochs.keys()) > 1:

    weighted_flux = 0 * epochs['Epoch 1']['FLUX']
    weighted_err  = 0 * epochs['Epoch 1']['FLUX']

    for ii in epochs:
        weighted_flux += epochs[ii]['FLUX'] / epochs[ii]['ERR']**2
        weighted_err  += 1 / epochs[ii]['ERR']**2

    weighted_flux /= weighted_err
    weighted_err  = np.sqrt(1/weighted_err)

else:
    weighted_flux = epochs['Epoch 1']['FLUX']
    weighted_err  = epochs['Epoch 1']['ERR']

In [ ]:
# Adjust the suitable wavelength range if needed
# The uploaded spectrum will have this wavelength range

wmin = 3700
wmax = 9200

In [ ]:
plt.figure(figsize=(9*np.sqrt(2), 9))

ax=plt.subplot(111)

mask = ''

for ii, epoch in enumerate(epochs):
    if epoch == 'Epoch 1':
        mask = np.where( (epochs[epoch]['WAVE'] >= wmin) & (epochs[epoch]['WAVE'] <= wmax))[0]

    temp_mask = np.where( (epochs[epoch]['WAVE'] >= wmin) & (epochs[epoch]['WAVE'] <= wmax))
    ax.plot(epochs[epoch]['WAVE'][temp_mask], epochs[epoch]['FLUX'][temp_mask], label=epoch)#, lw=8-3*ii)

ax.plot(epochs['Epoch 1']['WAVE'][mask], weighted_flux[mask], label='Weighted spec')
ax.plot(epochs['Epoch 1']['WAVE'][mask], weighted_err [mask], label='Weighted err')

ax.set_xlim(wmin, wmax)
#ax.set_xlim(6200, 6400)
ax.set_xlim(wmin, wmax)
ax.set_ylim(-0.1*np.nanmedian(epochs['Epoch 1']['FLUX']), 1.2*np.nanpercentile(epochs['Epoch 1']['FLUX'], q=95.15))
#ax.set_ylim(-0.1*np.nanmedian(epochs['Epoch 1']['FLUX']), 0.2e-16)

ax.legend(fontsize=legend_size)

plt.savefig(root + '{object}_{telescope}_{instrument}_{isot}.pdf'.format(object=comments['OBJECT'], telescope=comments['TELESCOPE'], instrument=comments['INSTRUMENT'], isot=comments['DATE-OBS']))

In [ ]:
# Write out stacked spectrum with the observational information

In [ ]:
output                  = table.Table(np.array([epochs['Epoch 1']['WAVE'], weighted_flux, weighted_err]).T, names=('WAVE', 'FLUX', 'FLUXERR'))
output.meta['comments'] = [key + ': ' +  str(comments[key]) for key in list(comments.keys())] + ['# Columns: WAVE FLUX FLUX_ERR']
output                  = output[mask] # Limits to useful wavelength range to wmin and wmax
output                  = output[np.logical_not(np.isnan(output['FLUX']))]
output                  = output[np.logical_not(np.isnan(output['FLUXERR']))]
output#[:5]

In [ ]:
root + '{object}_{telescope}_{instrument}_{isot}.ascii'.format(object=comments['OBJECT'], telescope=comments['TELESCOPE'], instrument=comments['INSTRUMENT'], isot=comments['DATE-OBS'])

In [ ]:
ascii.write(output, root + '{object}_{telescope}_{instrument}_{isot}.ascii'.format(object=comments['OBJECT'], telescope=comments['TELESCOPE'], instrument=comments['INSTRUMENT'], isot=comments['DATE-OBS']), format='no_header', overwrite=True)

In [ ]:
# Upload to Fritz

In [ ]:
# Add your token

In [ ]:
myToken = 'YOUR_TOKEN'

In [ ]:
present_groups = [x['id'] for x in get_groups(comments['OBJECT'])]
present_groups

In [ ]:
flag_submit = True

In [ ]:
file = open(root + '{object}_{telescope}_{instrument}_{isot}.ascii'.format(object=comments['OBJECT'], telescope=comments['TELESCOPE'], instrument=comments['INSTRUMENT'], isot=comments['DATE-OBS']))
root + '{object}_{telescope}_{instrument}_{isot}.ascii'.format(object=comments['OBJECT'], telescope=comments['TELESCOPE'], instrument=comments['INSTRUMENT'], isot=comments['DATE-OBS'])

In [ ]:
# User IDs
# Jesper: 23, Steve 39, Sean 1297, Priscila 1305

In [ ]:
data = {
"observed_by":   [23, 39],                                                                   # Jesper, Steve
"group_ids":     list([int(x) for x in np.union1d(present_groups, [41, 87, 1])]),            # Share with exisiting groups AND RCF, Jesper's group and Sitewise
"filename":      '{object}_{telescope}_{instrument}_{isot}.ascii'.format(object=comments['OBJECT'], telescope=comments['TELESCOPE'], instrument=comments['INSTRUMENT'], isot=comments['DATE-OBS']),
"reduced_by":    [39],                                                                       # Steve
"instrument_id": 26,                                                                         # ALFOSC
"observed_at":   comments['DATE-OBS'],
"obj_id":        comments['OBJECT'],                          
"ascii":         file.read(),
"wave_column": 0,
"flux_column": 1,
"fluxerr_column": 2,
"type": "source"  # SN = source, host = host
}

In [ ]:
[key + ': ' + str(data[key]) for key in data.keys() if key != 'ascii']

In [ ]:
if flag_submit:
    response = api('POST', 'https://fritz.science/api/spectrum/ascii', data=data)
    response